In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dt=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows=10_000_000)
train_dt.head()

In [ ]:
train_dt.shape

In [ ]:
train_dt.info()

In [ ]:
test_dt=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
test_dt.head()

In [ ]:
test_dt.info()

In [ ]:
train_dt.isna().sum()

In [ ]:
train_dt['Difference_longitude']=np.abs(np.asarray(train_dt['pickup_longitude']-train_dt['dropoff_longitude']))
train_dt['Difference_latitude']=np.abs(np.asarray(train_dt['pickup_latitude']-train_dt['dropoff_latitude']))


test_dt['Difference_longitude']=np.abs(np.asarray(test_dt['pickup_longitude']-test_dt['dropoff_longitude']))
test_dt['Difference_latitude']=np.abs(np.asarray(test_dt['pickup_latitude']-test_dt['dropoff_latitude']))

In [ ]:
print(f'Before Dropping null values: {len(train_dt)}')
train_dt.dropna(inplace=True)
print(f'After Dropping null values: {len(train_dt)}')

In [ ]:
plot = train_dt[:2000].plot.scatter('Difference_longitude', 'Difference_latitude')

In [ ]:
train_dt=train_dt[(train_dt['Difference_longitude']<5.0)&(train_dt['Difference_latitude']<5.0)]

In [ ]:
ls1=list(train_dt['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
train_dt['pickuptime']=ls1    



ls1=list(test_dt['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
test_dt['pickuptime']=ls1

In [ ]:
train_dt.head()

In [ ]:
ls1=list(train_dt['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:-4:]
    ls1[i]=pd.Timestamp(ls1[i])
    ls1[i]=ls1[i].weekday()
train_dt['Weekday']=ls1


ls1=list(test_dt['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:-4:]
    ls1[i]=pd.Timestamp(ls1[i])
    ls1[i]=ls1[i].weekday()
test_dt['Weekday']=ls1

In [ ]:
train_dt.head()

In [ ]:
test_dt.head()

In [ ]:
train_dt.drop('pickup_datetime',inplace=True,axis=1)
test_dt.drop('pickup_datetime',inplace=True,axis=1)

In [ ]:
train_dt['Weekday'].replace(to_replace=[i for i in range(0,7)],
                            value=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                              inplace=True)
test_dt['Weekday'].replace(to_replace=[i for i in range(0,7)],
                              value=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                              inplace=True)

In [ ]:
train_one_hot=pd.get_dummies(train_dt['Weekday'])
test_one_hot=pd.get_dummies(test_dt['Weekday'])
train_dt=pd.concat([train_dt,train_one_hot],axis=1)
test_dt=pd.concat([test_dt,test_one_hot],axis=1)

In [ ]:
train_dt.drop('Weekday',axis=1,inplace=True)
test_dt.drop('Weekday',axis=1,inplace=True)

In [ ]:
ls1=list(train_dt['pickuptime'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
train_dt['pickuptime']=ls1


ls1=list(test_dt['pickuptime'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
test_dt['pickuptime']=ls1

In [ ]:
train_dt.head()

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(train_dt['pickup_latitude']))
lon1 = np.asarray(np.radians(train_dt['pickup_longitude']))
lat2 = np.asarray(np.radians(train_dt['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_dt['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1=[] 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c

    
train_dt['Distance']=np.asarray(distance)*0.621



lat1 =np.asarray(np.radians(test_dt['pickup_latitude']))
lon1 = np.asarray(np.radians(test_dt['pickup_longitude']))
lat2 = np.asarray(np.radians(test_dt['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_dt['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
 
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c
test_dt['Distance']=np.asarray(distance)*0.621


In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(train_dt['pickup_latitude']))
lon1 = np.asarray(np.radians(train_dt['pickup_longitude']))
lat2 = np.asarray(np.radians(train_dt['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_dt['dropoff_longitude']))

lat3=np.zeros(len(train_dt))+np.radians(40.6413111)
lon3=np.zeros(len(train_dt))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
train_dt['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

train_dt['Dropoff_Distance_airport']=np.asarray(distance2)*0.621



lat1 =np.asarray(np.radians(test_dt['pickup_latitude']))
lon1 = np.asarray(np.radians(test_dt['pickup_longitude']))
lat2 = np.asarray(np.radians(test_dt['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_dt['dropoff_longitude']))

lat3=np.zeros(len(test_dt))+np.radians(40.6413111)
lon3=np.zeros(len(test_dt))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
test_dt['Pickup_Distance_airport']=np.asarray(distance1)*0.621


a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
test_dt['Dropoff_Distance_airport']=np.asarray(distance2)*0.621

In [ ]:
train_dt['Distance']=np.round(train_dt['Distance'],2)
train_dt['Pickup_Distance_airport']=np.round(train_dt['Pickup_Distance_airport'],2)
train_dt['Dropoff_Distance_airport']=np.round(train_dt['Dropoff_Distance_airport'],2)
test_dt['Distance']=np.round(test_dt['Distance'],2)
test_dt['Pickup_Distance_airport']=np.round(test_dt['Pickup_Distance_airport'],2)
test_dt['Dropoff_Distance_airport']=np.round(test_dt['Dropoff_Distance_airport'],2)

In [ ]:
train_dt.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)
test_dt.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

In [ ]:
train_dt['Difference_longitude']=np.abs(train_dt['Difference_longitude']-np.mean(train_dt['Difference_longitude']))
train_dt['Difference_longitude']=train_dt['Difference_longitude']/np.var(train_dt['Difference_longitude'])

In [ ]:
train_dt['Difference_latitude']=np.abs(train_dt['Difference_latitude']-np.mean(train_dt['Difference_latitude']))
train_dt['Difference_latitude']=train_dt['Difference_latitude']/np.var(train_dt['Difference_latitude'])

In [ ]:
test_dt['Difference_longitude']=np.abs(test_dt['Difference_longitude']-np.mean(test_dt['Difference_longitude']))
test_dt['Difference_longitude']=test_dt['Difference_longitude']/np.var(test_dt['Difference_longitude'])

test_dt['Difference_latitude']=np.abs(test_dt['Difference_latitude']-np.mean(test_dt['Difference_latitude']))
test_dt['Difference_latitude']=test_dt['Difference_latitude']/np.var(test_dt['Difference_latitude'])

In [ ]:
train_dt.shape

In [ ]:
test_dt.shape

In [ ]:
from sklearn.model_selection import train_test_split
X=train_dt.drop(['key','fare_amount'],axis=1)
y=train_dt['fare_amount']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01,random_state=80)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)
lr.fit(X_train,y_train)
print(lr.score(X_test,y_test))

In [ ]:
pred=np.round(lr.predict(test_dt.drop('key',axis=1)),2)

In [ ]:
pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv').head()

In [ ]:
Submission=pd.DataFrame(data=pred,columns=['fare_amount'])
Submission['key']=test_dt['key']
Submission=Submission[['key','fare_amount']]

In [ ]:
Submission.set_index('key',inplace=True)

In [ ]:
Submission.to_csv('Submission.csv')